## Limpar dados

Esse notebook está responsável por limpar os dados baixados. Para isso, são feitas algumas investigações para entender melhor os dados e ajudar a definir qual abordagem de limpeza utilizar para cada caso.

---

In [1]:
# importando pacotes
import os
import pandas as pd
import numpy as np

In [2]:
# agora que temos os dados armazenados, podemos criar o nosso dataframe e seguir com as análises e limpeza
pasta_atual = os.getcwd()
pasta_pai = os.path.dirname(pasta_atual)
pasta_data = os.path.join(pasta_pai, "data")

df = pd.read_csv(f"{pasta_data}\\dados_brutos.csv", parse_dates=["date"])\
    .sort_values(by=["date", "appId"])\
    .reset_index(drop=True)

In [3]:
df

,appId,date,dauReal,mauReal,country,lang,predictionLoss,newinstalls,category,ratings,daily_ratings,reviews,daily_reviews
0,com.app.68335,1912-02-01,8627.0,64260.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,com.app.47280,1912-02-02,9695892.0,21586101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,com.app.22262,1912-02-03,97263.0,582518.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,com.app.64353,1912-02-04,269402.0,701284.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,com.app.92275,1912-02-05,69026.0,544407.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54077,NaN,2220-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54078,com.app.39058,2220-07-30,15750.0,106200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54079,NaN,2220-07-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54080,com.app.27567,2220-07-31,6728.0,91481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Vamos começar olhando uma visão geral sobre os valores nulos do dataframe
df.isna().sum()

appId                28
date                  0
dauReal           12867
mauReal           12693
country             570
lang                570
predictionLoss      570
newinstalls        5782
category           6605
ratings            6605
daily_ratings      6605
reviews            6605
daily_reviews      6605
dtype: int64

In [5]:
# Valores de appId nulos não nos interessam, então vamos começar descartando eles:
df = df.loc[~df["appId"].isna()]

In [6]:
df.isna().sum()

appId                 0
date                  0
dauReal           12839
mauReal           12665
country             542
lang                542
predictionLoss      542
newinstalls        5754
category           6577
ratings            6577
daily_ratings      6577
reviews            6577
daily_reviews      6577
dtype: int64

In [7]:
# Vamos ver agora se todos os valores restantes apresentam o mesmo padrão
regex = r"^com\.app\.\d+$"
df.loc[~df["appId"].str.match(regex, na=False), "appId"]

Series([], Name: appId, dtype: object)

In [8]:
# Todos os valores de appId apresentam o mesmo padrão e não apresentam valores nulos, podemos seguir em frente.

In [9]:
# Não existem valores nulo de data, mas isso não significa que não podem haver valores inválidos
sorted(df["date"].dt.year.unique())

[np.int32(1912),
 np.int32(1980),
 np.int32(2024),
 np.int32(2044),
 np.int32(2220)]

In [10]:
# Todos os anos exceto 2024 são anos inválidos e podem ser filtrados
df = df.loc[df["date"].dt.year == 2024]

In [11]:
df.isna().sum()

appId                 0
date                  0
dauReal           12839
mauReal           12665
country             469
lang                469
predictionLoss      469
newinstalls        5681
category           6504
ratings            6504
daily_ratings      6504
reviews            6504
daily_reviews      6504
dtype: int64

In [12]:
# ver se existem combinações repetidas de appId e date
df[df.duplicated(subset=["appId", "date"])]

,appId,date,dauReal,mauReal,country,lang,predictionLoss,newinstalls,category,ratings,daily_ratings,reviews,daily_reviews
34,com.app.15493,2024-01-01,52117.0,546565.0,br,pt,1553.0,170.0,FINANCE,14778.0,10.0,6388.0,0.0
227,com.app.15493,2024-01-02,66132.0,553051.0,br,pt,1403.0,170.0,FINANCE,14788.0,8.0,6388.0,0.0
420,com.app.15493,2024-01-03,67342.0,559060.0,br,pt,1419.0,170.0,FINANCE,14796.0,7.0,6388.0,0.0
613,com.app.15493,2024-01-04,62500.0,564274.0,br,pt,1401.0,170.0,FINANCE,14803.0,2.0,6387.0,3.0
806,com.app.15493,2024-01-05,59688.0,567906.0,br,pt,1415.0,170.0,FINANCE,14805.0,4.0,6390.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44556,com.app.97802,2024-08-19,6605.0,69105.0,br,pt,10553.0,199.0,OTHERS,2925.0,1.0,1268.0,2.0
44558,com.app.97976,2024-08-19,8810.0,103498.0,br,pt,4871.0,954.0,SHOPPING,10856.0,15.0,3604.0,2.0
44560,com.app.97988,2024-08-19,3187.0,33252.0,br,pt,1163.0,109.0,SHOPPING,11074.0,0.0,4273.0,0.0
44562,com.app.98198,2024-08-19,NaN,NaN,br,pt,1911.0,23.0,OTHERS,4071.0,0.0,2240.0,0.0


In [13]:
# deletar duplicatas
df = df.drop_duplicates()

In [14]:
# vamos ver se ainda existem combinações repetidas de appId e date depois de deletar linhas duplicadas
df[df.duplicated(subset=["appId", "date"])]

,appId,date,dauReal,mauReal,country,lang,predictionLoss,newinstalls,category,ratings,daily_ratings,reviews,daily_reviews


In [15]:
# como não temos casos de dados de um appId mais de uma vez no mesmo dia, para auxiliar nas análises futuras, vamos garantir
# que para cada dia temos todas as combinações de appId e data
quantidade_original = df.shape[0]

periodo_completo = pd.date_range(start=df["date"].min(), end=df["date"].max())

app_ids = df["appId"].unique()

todas_combinacoes = pd.MultiIndex.from_product([app_ids, periodo_completo], names=["appId", "date"])

df = df.set_index(["appId", "date"]).reindex(todas_combinacoes).reset_index()
print(f"Dados adicionados: {df.shape[0] - quantidade_original}")

Dados adicionados: 1528


In [16]:
# Vamos seguir olhando as columnas "country" e "lang"
print(f"valores distintos 'country': {df["country"].nunique()}")
print(f"valores distintos 'lang': {df["lang"].nunique()}")

valores distintos 'country': 1
valores distintos 'lang': 1


In [17]:
# ambas as colunas apresentam apenas um único valor, portanto essas colunas não agregam valor e podem ser descartadas
df = df.drop(["country", "lang"], axis=1)

In [18]:
# Vamos entender a coluna "category".
# Primeiro, vamos ver se existem casos em que um aplicativo pode ter mais de uma categoria
df.groupby("appId")["category"]\
    .nunique()\
    .reset_index(name = "contagem_categorias")\
    .query("contagem_categorias >= 2")\
    .sort_values(by = "contagem_categorias", ascending = False)

,appId,contagem_categorias


In [19]:
# Não existem casos de um aplicativo ter mais de uma categoria, mas vamos ver se existem aplicativos com
# uma categoria definida e também com valores nulos
df.groupby("appId")["category"]\
    .nunique(dropna=False)\
    .reset_index(name = "contagem_categorias")\
    .query("contagem_categorias >= 2")\
    .sort_values(by = "contagem_categorias", ascending = False)

,appId,contagem_categorias
0,com.app.10626,2
2,com.app.13071,2
3,com.app.13421,2
4,com.app.13655,2
6,com.app.14509,2
...,...,...
189,com.app.97802,2
190,com.app.97976,2
191,com.app.97988,2
192,com.app.98198,2


In [20]:
# Esse valor foi populado pela tabela "ratings_reviews", e provavelmente se refere a categoria do review.
# Entretanto, como observamos apenas valores válidos exclusivos por aplicativo, podemos atribuir a categoria
# ao aplicativo independente de ter ocorrido uma avaliação.
mapa = df.dropna(subset=["category"]).drop_duplicates("appId").set_index("appId")["category"].to_dict()

df["category"] = df.apply(
    lambda linha: mapa.get(linha["appId"], "NO_CATEGORY") if pd.isna(linha["category"]) else linha["category"],
    axis=1
)

In [21]:
df.isna().sum()

appId                 0
date                  0
dauReal           14334
mauReal           14160
predictionLoss     1997
newinstalls        7206
category              0
ratings            8030
daily_ratings      8030
reviews            8030
daily_reviews      8030
dtype: int64

In [22]:
# não existem mais valores nulos para "Category", vamos validar se existem casos de apps com mais de uma categoria
df.groupby("appId")["category"]\
    .nunique()\
    .reset_index(name = "contagem_categorias")\
    .query("contagem_categorias >= 2")\
    .sort_values(by = "contagem_categorias", ascending = False)

,appId,contagem_categorias


In [23]:
# as colunas "ratings" e "reviews" são cumulativas e, portanto, nunca devem apresentar valor nulo (exceto
# a primeira ocorrencia).
# A princípio, vamos preencher esses valores, mas precisamos tomar cuidado pois também existem as colunas "daily_ratings"
# e "daily_reviews" que são relacionadas.
df.loc[
    (df["ratings"].isna() ^ df["daily_ratings"].isna())
    | (df["reviews"].isna() ^ df["daily_reviews"].isna())
]

,appId,date,dauReal,mauReal,predictionLoss,newinstalls,category,ratings,daily_ratings,reviews,daily_reviews


In [24]:
# não existem casos em que o valor diário está nulo mas o cumulativo está definido, e vice versa.

In [25]:
# quando existir valores nulos de "ratings" e "reviews", vamos atribuir que não teve novas avaliações nesse período,
# e preencher com o ultimo valor observado
df[["ratings", "reviews"]] = df.groupby("appId")[["ratings", "reviews"]]\
      .transform(lambda x: x.ffill())

In [26]:
df.isna().sum()

appId                 0
date                  0
dauReal           14334
mauReal           14160
predictionLoss     1997
newinstalls        7206
category              0
ratings            4628
daily_ratings      8030
reviews            4628
daily_reviews      8030
dtype: int64

In [27]:
# Ainda restam os casos em que "reviews" e "ratings" começam com valor nulo (sem dados para serem propagados)
# Entretanto, não sabemos como era o comportamento dessas avaliações antes do primeiro dado armazenado.
# Vamos adotar que antes da primeira avaliação, não haviam avaliações
df[["ratings", "reviews"]] = df[["ratings", "reviews"]].fillna(0)

In [28]:
df.isna().sum()

appId                 0
date                  0
dauReal           14334
mauReal           14160
predictionLoss     1997
newinstalls        7206
category              0
ratings               0
daily_ratings      8030
reviews               0
daily_reviews      8030
dtype: int64

In [29]:
# vamos recalcular as colunas de "daily_ratings" e "daily_reviews", presumindo que o banco de dados é atualizado consultando 
# a quantidade de avaliações diárias, pois é uma informação mais garantida que apenas olhar para a quantiadde de novas
# avaliações 
df["daily_ratings"] = df.groupby("appId")["ratings"].shift(-1) - df["ratings"] 
df["daily_reviews"] = df.groupby("appId")["reviews"].shift(-1) - df["reviews"] 


In [30]:
df.isna().sum()

appId                 0
date                  0
dauReal           14334
mauReal           14160
predictionLoss     1997
newinstalls        7206
category              0
ratings               0
daily_ratings       195
reviews               0
daily_reviews       195
dtype: int64

In [31]:
# ainda sobram os casos em que daily_ratings e daily_reviews são nulos, pois não temos o valor das avaliações seguintes.
# para esses casos, vamos replicar o último valor observado
df[["daily_ratings", "daily_reviews"]] = df.groupby("appId")[["daily_ratings", "daily_reviews"]]\
      .transform(lambda x: x.ffill())

In [32]:
df.isna().sum()

appId                 0
date                  0
dauReal           14334
mauReal           14160
predictionLoss     1997
newinstalls        7206
category              0
ratings               0
daily_ratings         0
reviews               0
daily_reviews         0
dtype: int64

In [33]:
# para os valores restantes, existem alguns casos que podem explicar valores nulos, vamos adotar a seguinte regra:
# para mauReal e predictionLoss, vamos interpolar os dados possíveis, pois é mais improvável que tenhamos 30 dias
# sem usuários, e um período sem nenhuma desinstalação
df[["mauReal", "predictionLoss"]] = df[["mauReal", "predictionLoss"]].interpolate(method="linear")

In [34]:
df.isna().sum()

appId                 0
date                  0
dauReal           14334
mauReal              82
predictionLoss        0
newinstalls        7206
category              0
ratings               0
daily_ratings         0
reviews               0
daily_reviews         0
dtype: int64

In [35]:
# não existem mais dados nulos de predictionLoss, mas ainda existem dados nulos de mauReal
# Replicar os dados finais para mauReal
df["mauReal"] = df.groupby("appId")["mauReal"]\
    .transform(lambda x: x.ffill())

In [36]:
df.isna().sum()

appId                 0
date                  0
dauReal           14334
mauReal              82
predictionLoss        0
newinstalls        7206
category              0
ratings               0
daily_ratings         0
reviews               0
daily_reviews         0
dtype: int64

In [37]:
# dados finais não foram replicados, vamos investigar os dados que mauReal é zero
df.loc[df["mauReal"].isna(), "appId"].unique()

array(['com.app.10626'], dtype=object)

In [38]:
# esse erro acontece apenas com um único aplicativo
df.loc[df["appId"] == "com.app.10626"]

,appId,date,dauReal,mauReal,predictionLoss,newinstalls,category,ratings,daily_ratings,reviews,daily_reviews
0,com.app.10626,2024-01-01,NaN,NaN,30372.000000,4064.0,FINANCE,418299.0,210.0,111173.0,88.0
1,com.app.10626,2024-01-02,NaN,NaN,34775.000000,4065.0,FINANCE,418509.0,426.0,111261.0,86.0
2,com.app.10626,2024-01-03,NaN,NaN,36231.000000,4066.0,FINANCE,418935.0,496.0,111347.0,111.0
3,com.app.10626,2024-01-04,NaN,NaN,36835.000000,4066.0,FINANCE,419431.0,495.0,111458.0,120.0
4,com.app.10626,2024-01-05,NaN,NaN,36060.000000,4066.0,FINANCE,419926.0,554.0,111578.0,93.0
...,...,...,...,...,...,...,...,...,...,...,...
278,com.app.10626,2024-10-05,NaN,1.912925e+06,25166.000000,6098.0,FINANCE,546217.0,0.0,135414.0,0.0
279,com.app.10626,2024-10-06,NaN,1.887118e+06,26743.000000,4851.0,FINANCE,546217.0,0.0,135414.0,0.0
280,com.app.10626,2024-10-07,NaN,1.861310e+06,28874.000000,4895.0,FINANCE,546217.0,0.0,135414.0,0.0
281,com.app.10626,2024-10-08,NaN,1.835503e+06,19844.666667,4940.0,FINANCE,546217.0,0.0,135414.0,0.0


In [39]:
# esse aplicativo apresenta um valor alto de usuários desde o inicio.
# como encontramos valores de rating e reviews desde o início, vamos presumir que o app já havia sido lançado antes
# de 2024-03-23
df["mauReal"] = df.groupby("appId")["mauReal"]\
    .transform(lambda x: x.bfill())

In [40]:
df.isna().sum()

appId                 0
date                  0
dauReal           14334
mauReal               0
predictionLoss        0
newinstalls        7206
category              0
ratings               0
daily_ratings         0
reviews               0
daily_reviews         0
dtype: int64

In [41]:
# Não sabemos como o banco de dados define a informação e "newinstalls", entretanto, é um tipo de informação que esperamos
# ter mais formas de conseguir essa informação correta. Ou seja, caso ela não exista, vamos presumir que ela não exista
# pois não ocorreram novas isntalações
df["newinstalls"] = df["newinstalls"].fillna(0)

In [42]:
df.isna().sum()

appId                 0
date                  0
dauReal           14334
mauReal               0
predictionLoss        0
newinstalls           0
category              0
ratings               0
daily_ratings         0
reviews               0
daily_reviews         0
dtype: int64

In [43]:
# Por fim, vamos analisar DAU.
# Não sabemos como essa informação é calculada, mas diferente de "newisntalls", existem mais situações em que podem ocorrer
# erros.
# Vamos então interpolar os dados sempre que mauReal for maior que zero
filtro = df["mauReal"] > 0
df.loc[filtro, "dauReal"] = df["dauReal"].where(filtro).interpolate()

In [44]:
df.isna().sum()

appId              0
date               0
dauReal           82
mauReal            0
predictionLoss     0
newinstalls        0
category           0
ratings            0
daily_ratings      0
reviews            0
daily_reviews      0
dtype: int64

In [45]:
# quase todos os valores foram corrigidos, vamos observar os valores restantes
df.loc[df["dauReal"].isna(), "appId"].unique()

array(['com.app.10626'], dtype=object)

In [46]:
# localizamos o mesmo caso anterior
df.loc[df["appId"] == "com.app.10626"]

,appId,date,dauReal,mauReal,predictionLoss,newinstalls,category,ratings,daily_ratings,reviews,daily_reviews
0,com.app.10626,2024-01-01,NaN,1.950503e+06,30372.000000,4064.0,FINANCE,418299.0,210.0,111173.0,88.0
1,com.app.10626,2024-01-02,NaN,1.950503e+06,34775.000000,4065.0,FINANCE,418509.0,426.0,111261.0,86.0
2,com.app.10626,2024-01-03,NaN,1.950503e+06,36231.000000,4066.0,FINANCE,418935.0,496.0,111347.0,111.0
3,com.app.10626,2024-01-04,NaN,1.950503e+06,36835.000000,4066.0,FINANCE,419431.0,495.0,111458.0,120.0
4,com.app.10626,2024-01-05,NaN,1.950503e+06,36060.000000,4066.0,FINANCE,419926.0,554.0,111578.0,93.0
...,...,...,...,...,...,...,...,...,...,...,...
278,com.app.10626,2024-10-05,256686.455696,1.912925e+06,25166.000000,6098.0,FINANCE,546217.0,0.0,135414.0,0.0
279,com.app.10626,2024-10-06,253335.746835,1.887118e+06,26743.000000,4851.0,FINANCE,546217.0,0.0,135414.0,0.0
280,com.app.10626,2024-10-07,249985.037975,1.861310e+06,28874.000000,4895.0,FINANCE,546217.0,0.0,135414.0,0.0
281,com.app.10626,2024-10-08,246634.329114,1.835503e+06,19844.666667,4940.0,FINANCE,546217.0,0.0,135414.0,0.0


In [47]:
# nesse caso, o aplicativo apresenta ter muitos usuários mesmo antes do dia 2024-03-23. Vamos então preencher esses dados
# com o primeiro dado disponível
df["dauReal"] = df.groupby("appId")["dauReal"]\
    .transform(lambda x: x.bfill())

In [48]:
df.isna().sum()

appId             0
date              0
dauReal           0
mauReal           0
predictionLoss    0
newinstalls       0
category          0
ratings           0
daily_ratings     0
reviews           0
daily_reviews     0
dtype: int64

In [49]:
# agora que não existem mais nenhum valor nulo, vamos ver se todos os valores fazem sentido
df.describe()

,date,dauReal,mauReal,predictionLoss,newinstalls,ratings,daily_ratings,reviews,daily_reviews
count,55185,5.518500e+04,5.518500e+04,55185.000000,55185.000000,5.518500e+04,55185.000000,5.518500e+04,55185.000000
mean,2024-05-21 00:00:00,4.095747e+05,1.688491e+06,11719.263595,3578.492000,3.248394e+05,129.885059,8.370502e+04,19.599384
min,2024-01-01 00:00:00,7.500000e+02,7.500000e+02,331.000000,0.000000,0.000000e+00,-4852.000000,0.000000e+00,-2252.000000
25%,2024-03-11 00:00:00,9.255140e+03,6.865100e+04,1758.000000,101.000000,3.889000e+03,0.000000,1.296000e+03,0.000000
50%,2024-05-21 00:00:00,3.344800e+04,1.981270e+05,3143.000000,623.000000,3.428600e+04,7.000000,8.406000e+03,2.000000
75%,2024-07-31 00:00:00,1.616113e+05,7.610518e+05,8588.000000,2363.000000,1.655280e+05,52.000000,4.676800e+04,9.000000
max,2024-10-09 00:00:00,1.064815e+07,2.960584e+07,189683.000000,73886.000000,1.323622e+07,52712.000000,1.491128e+06,22965.000000
std,NaN,1.236185e+06,4.390673e+06,23396.419519,8389.611197,1.163578e+06,631.850599,2.227374e+05,148.555477


In [50]:
# parece que todos os valores estão corretos, e como não temos informações do contexto real dos aplicativos, não sabemos
# se a ordem de grandeza faz sentido, então vamos presumir que os dados estão na ordem de grandeza correta.
# Entretanto, os valores de "daily_ratings" e "daily_reviews" apresentam um valor mínimo muito menor que o esperado.
df.loc[
    (df["daily_ratings"] < -1000)
    | (df["daily_reviews"] < -1000)
]

,appId,date,dauReal,mauReal,predictionLoss,newinstalls,category,ratings,daily_ratings,reviews,daily_reviews
8023,com.app.22262,2024-04-09,8.844700e+04,5.924690e+05,1799.0,1787.0,FINANCE,95243.0,-1012.0,19023.0,-112.0
10528,com.app.28498,2024-02-27,2.768869e+05,3.992835e+05,80830.0,33375.0,FOOD_AND_DRINK,13190883.0,-2774.0,1454409.0,-116.0
10529,com.app.28498,2024-02-28,2.809939e+05,4.040532e+05,78956.0,31069.0,FOOD_AND_DRINK,13188109.0,-4852.0,1454293.0,-227.0
10623,com.app.28498,2024-06-01,5.343237e+05,3.426663e+06,79788.0,31949.0,FOOD_AND_DRINK,13233651.0,-4040.0,1461017.0,-592.0
21000,com.app.45088,2024-02-28,4.238240e+06,1.325608e+07,76175.0,22158.0,FINANCE,2739876.0,-1528.0,469645.0,-154.0
21041,com.app.45088,2024-04-09,4.667966e+06,1.337748e+07,65913.0,20640.0,FINANCE,2835627.0,3284.0,479055.0,-2252.0
26754,com.app.59849,2024-06-01,1.237007e+06,2.195505e+06,17035.0,5377.0,OTHERS,1524417.0,-1184.0,377545.0,-568.0
43451,com.app.87362,2024-06-01,1.991950e+05,7.212820e+05,26805.0,14282.0,OTHERS,672870.0,-2118.0,260255.0,-1039.0


In [51]:
# Olhando os dados, os dados não parecem ser discrepantes devido a grande quantidade de usuários envolvidos,
# o que torna mais possível que uma quantidade significativa de usuários retirem as suas avaliações

In [52]:
# Agora que todos os dados estão limpos, podemos armazenar o dataframe
pasta_atual = os.getcwd()
pasta_pai = os.path.dirname(pasta_atual)
pasta_data = os.path.join(pasta_pai, "data")

df.to_csv(f"{pasta_data}\\dados_limpos.csv", index=False)